In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('UNSW_NB15_train-set.csv').drop(['id'], axis=1)


In [18]:
import keras
from keras.models import Sequential   # importing Sequential model
from keras.layers import Dense        # importing Dense layers
from keras.wrappers.scikit_learn import KerasClassifier


In [16]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import RobustScaler

In [10]:
# separate categroicals with non categoricals 
output1 = train_df[['attack_cat']].copy()  # multiclass
output2 = train_df[['label']].copy()       # binary

input_categroicals = train_df.dtypes[train_df.dtypes == object].index.tolist()[:-1]
input_numericals = train_df.dtypes[train_df.dtypes != object].index.tolist()[:-1]

input_categorical_values = train_df[input_categroicals].copy()
input_numerical_values = train_df[input_numericals].copy()

In [12]:
# encoding the values
output1 = pd.get_dummies(output1)

rbs= RobustScaler()
rbs.fit(input_numerical_values)
input_numerical_values = pd.DataFrame(rbs.transform(input_numerical_values), columns=input_numericals)
input_numerical_values

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,0.179469,0.4,0.2,-0.131902,0.007260,-0.025220,-0.010417,0.892857,-0.009738,0.253129,...,-0.166667,0.00,0.0,-0.181818,0.0,0.0,0.0,-0.285714,-0.3,0.0
1,0.970450,1.2,3.6,0.233129,37.976407,-0.025185,-1.000000,0.884921,-0.009803,18.032918,...,-0.166667,0.00,0.0,-0.090909,0.0,0.0,0.0,-0.285714,0.2,0.0
2,2.427244,0.6,1.4,-0.050613,11.816697,-0.025700,-1.000000,0.884921,-0.009880,2.136200,...,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,-0.142857,0.2,0.0
3,2.514830,1.0,1.0,0.151840,0.549909,-0.025704,-1.000000,0.884921,-0.009867,0.068652,...,0.000000,0.00,0.0,0.000000,1.0,1.0,0.0,-0.142857,-0.3,0.0
4,0.670406,0.8,0.4,0.079755,0.094374,-0.025546,0.000000,0.884921,-0.009801,0.091221,...,0.000000,0.25,0.0,3.363636,0.0,0.0,0.0,-0.142857,3.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,-0.002355,0.0,-0.2,-0.242331,-0.148820,0.863309,0.000000,-0.115079,0.560186,-0.051967,...,3.666667,5.75,6.0,1.909091,0.0,0.0,0.0,3.000000,2.0,0.0
175337,0.754692,0.8,0.6,0.145706,0.172414,-0.025544,0.000000,0.884921,-0.009798,0.124133,...,-0.166667,0.00,0.0,-0.090909,0.0,0.0,0.0,-0.285714,-0.3,0.0
175338,-0.002355,0.0,-0.2,-0.242331,-0.148820,0.863309,0.000000,-0.115079,0.560186,-0.051967,...,0.166667,0.50,1.0,0.909091,0.0,0.0,0.0,0.000000,0.8,0.0
175339,-0.002355,0.0,-0.2,-0.242331,-0.148820,0.863309,0.000000,-0.115079,0.560186,-0.051967,...,4.666667,7.25,6.5,2.454545,0.0,0.0,0.0,3.857143,2.6,0.0


In [38]:
(train_df['attack_cat'].value_counts() / train_df.shape[0]).round(4) * 100

Normal            31.94
Generic           22.81
Exploits          19.04
Fuzzers           10.37
DoS                6.99
Reconnaissance     5.98
Analysis           1.14
Backdoor           1.00
Shellcode          0.65
Worms              0.07
Name: attack_cat, dtype: float64

In [39]:
output1

,attack_cat_Analysis,attack_cat_Backdoor,attack_cat_DoS,attack_cat_Exploits,attack_cat_Fuzzers,attack_cat_Generic,attack_cat_Normal,attack_cat_Reconnaissance,attack_cat_Shellcode,attack_cat_Worms
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
175336,0,0,0,0,0,1,0,0,0,0
175337,0,0,0,0,0,0,0,0,1,0
175338,0,0,0,0,0,1,0,0,0,0
175339,0,0,0,0,0,1,0,0,0,0


In [30]:
model = Sequential()
model.add(Dense(39, input_dim=39, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 39)                1560      
                                                                 
 dense_31 (Dense)            (None, 10)                400       
                                                                 
Total params: 1,960
Trainable params: 1,960
Non-trainable params: 0
_________________________________________________________________


In [31]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 11s 2ms/step - loss: 46.9541 - accuracy: 0.6934
Epoch 2/10
5480/5480 [==============================] - 10s 2ms/step - loss: 43.3682 - accuracy: 0.7486
Epoch 3/10
5480/5480 [==============================] - 11s 2ms/step - loss: 30.3333 - accuracy: 0.7603
Epoch 4/10
5480/5480 [==============================] - 12s 2ms/step - loss: 27.7791 - accuracy: 0.7678
Epoch 5/10
5480/5480 [==============================] - 11s 2ms/step - loss: 19.6002 - accuracy: 0.7703
Epoch 6/10
5480/5480 [==============================] - 14s 3ms/step - loss: 13.5532 - accuracy: 0.7757
Epoch 7/10
5480/5480 [==============================] - 11s 2ms/step - loss: 10.1757 - accuracy: 0.7788
Epoch 8/10
5480/5480 [==============================] - 11s 2ms/step - loss: 7.2289 - accuracy: 0.7813
Epoch 9/10
5480/5480 [==============================] - 10s 2ms/step - loss: 1.8347 - accuracy: 0.7779
Epoch 10/10
5480/5480 [==============================] - 10s 2ms/s

In [32]:
model = Sequential()
model.add(Dense(26, input_dim=39, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 26)                1040      
                                                                 
 dense_33 (Dense)            (None, 10)                270       
                                                                 
Total params: 1,310
Trainable params: 1,310
Non-trainable params: 0
_________________________________________________________________


In [33]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 11s 2ms/step - loss: 40.0290 - accuracy: 0.6936
Epoch 2/10
5480/5480 [==============================] - 10s 2ms/step - loss: 27.4622 - accuracy: 0.7396
Epoch 3/10
5480/5480 [==============================] - 10s 2ms/step - loss: 28.7468 - accuracy: 0.7481
Epoch 4/10
5480/5480 [==============================] - 11s 2ms/step - loss: 29.0116 - accuracy: 0.7541
Epoch 5/10
5480/5480 [==============================] - 10s 2ms/step - loss: 27.7951 - accuracy: 0.7605
Epoch 6/10
5480/5480 [==============================] - 10s 2ms/step - loss: 19.0058 - accuracy: 0.7646
Epoch 7/10
5480/5480 [==============================] - 10s 2ms/step - loss: 19.2734 - accuracy: 0.7662
Epoch 8/10
5480/5480 [==============================] - 12s 2ms/step - loss: 13.8842 - accuracy: 0.7690
Epoch 9/10
5480/5480 [==============================] - 11s 2ms/step - loss: 10.5784 - accuracy: 0.7747
Epoch 10/10
5480/5480 [==============================] - 11s 2ms

In [34]:
model = Sequential()
model.add(Dense(39, input_dim=39, activation='relu'))
model.add(Dense(26, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 39)                1560      
                                                                 
 dense_35 (Dense)            (None, 26)                1040      
                                                                 
 dense_36 (Dense)            (None, 10)                270       
                                                                 
Total params: 2,870
Trainable params: 2,870
Non-trainable params: 0
_________________________________________________________________


In [35]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 11s 2ms/step - loss: 29.1444 - accuracy: 0.7162
Epoch 2/10
5480/5480 [==============================] - 12s 2ms/step - loss: 0.6614 - accuracy: 0.7605
Epoch 3/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5918 - accuracy: 0.7700
Epoch 4/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5688 - accuracy: 0.7737
Epoch 5/10
5480/5480 [==============================] - 12s 2ms/step - loss: 0.5579 - accuracy: 0.7765
Epoch 6/10
5480/5480 [==============================] - 12s 2ms/step - loss: 0.5487 - accuracy: 0.7792
Epoch 7/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5424 - accuracy: 0.7801
Epoch 8/10
5480/5480 [==============================] - 11s 2ms/step - loss: 0.5382 - accuracy: 0.7821
Epoch 9/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5316 - accuracy: 0.7830
Epoch 10/10
5480/5480 [==============================] - 11s 2ms/step - 

In [41]:
model = Sequential()
model.add(Dense(39, input_dim=39, activation='relu'))
model.add(Dense(26, activation='relu'))
model.add(Dense(17, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 39)                1560      
                                                                 
 dense_41 (Dense)            (None, 26)                1040      
                                                                 
 dense_42 (Dense)            (None, 17)                459       
                                                                 
 dense_43 (Dense)            (None, 10)                180       
                                                                 
Total params: 3,239
Trainable params: 3,239
Non-trainable params: 0
_________________________________________________________________


In [42]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 45s 8ms/step - loss: 3.5510 - accuracy: 0.7194
Epoch 2/10
5480/5480 [==============================] - 60s 11ms/step - loss: 0.7289 - accuracy: 0.7640
Epoch 3/10
5480/5480 [==============================] - 78s 14ms/step - loss: 0.5819 - accuracy: 0.7698
Epoch 4/10
5480/5480 [==============================] - 109s 20ms/step - loss: 0.5689 - accuracy: 0.7723
Epoch 5/10
5480/5480 [==============================] - 46s 8ms/step - loss: 0.5692 - accuracy: 0.7765
Epoch 6/10
5480/5480 [==============================] - 12s 2ms/step - loss: 0.5500 - accuracy: 0.7774
Epoch 7/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5491 - accuracy: 0.7794
Epoch 8/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5396 - accuracy: 0.7795
Epoch 9/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5359 - accuracy: 0.7810
Epoch 10/10
5480/5480 [==============================] - 9s 2ms/step 

In [43]:
model = Sequential()
model.add(Dense(39, input_dim=39, activation='relu'))
model.add(Dense(17, activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 39)                1560      
                                                                 
 dense_45 (Dense)            (None, 17)                680       
                                                                 
 dense_46 (Dense)            (None, 11)                198       
                                                                 
 dense_47 (Dense)            (None, 10)                120       
                                                                 
Total params: 2,558
Trainable params: 2,558
Non-trainable params: 0
_________________________________________________________________


In [44]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 11s 2ms/step - loss: 6.7781 - accuracy: 0.7084
Epoch 2/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.6442 - accuracy: 0.7562
Epoch 3/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.6005 - accuracy: 0.7663
Epoch 4/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5821 - accuracy: 0.7694
Epoch 5/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5694 - accuracy: 0.7732
Epoch 6/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5637 - accuracy: 0.7733
Epoch 7/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5526 - accuracy: 0.7770
Epoch 8/10
5480/5480 [==============================] - 12s 2ms/step - loss: 0.5643 - accuracy: 0.7776
Epoch 9/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5571 - accuracy: 0.7779
Epoch 10/10
5480/5480 [==============================] - 10s 2ms/step - l

In [45]:
model = Sequential()
model.add(Dense(39, input_dim=39, activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 39)                1560      
                                                                 
 dense_49 (Dense)            (None, 11)                440       
                                                                 
 dense_50 (Dense)            (None, 7)                 84        
                                                                 
 dense_51 (Dense)            (None, 10)                80        
                                                                 
Total params: 2,164
Trainable params: 2,164
Non-trainable params: 0
_________________________________________________________________


In [46]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 11s 2ms/step - loss: 4.2997 - accuracy: 0.6645
Epoch 2/10
5480/5480 [==============================] - 11s 2ms/step - loss: 0.7762 - accuracy: 0.7196
Epoch 3/10
5480/5480 [==============================] - 11s 2ms/step - loss: 0.6966 - accuracy: 0.7420
Epoch 4/10
5480/5480 [==============================] - 11s 2ms/step - loss: 0.6293 - accuracy: 0.7538
Epoch 5/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.6094 - accuracy: 0.7646
Epoch 6/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5951 - accuracy: 0.7673
Epoch 7/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5885 - accuracy: 0.7695
Epoch 8/10
5480/5480 [==============================] - 9s 2ms/step - loss: 0.5813 - accuracy: 0.7722
Epoch 9/10
5480/5480 [==============================] - 9s 2ms/step - loss: 0.5791 - accuracy: 0.7730
Epoch 10/10
5480/5480 [==============================] - 10s 2ms/step - los

In [47]:
model = Sequential()
model.add(Dense(17, input_dim=39, activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 17)                680       
                                                                 
 dense_53 (Dense)            (None, 11)                198       
                                                                 
 dense_54 (Dense)            (None, 7)                 84        
                                                                 
 dense_55 (Dense)            (None, 10)                80        
                                                                 
Total params: 1,042
Trainable params: 1,042
Non-trainable params: 0
_________________________________________________________________


In [48]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 11s 2ms/step - loss: 8.7483 - accuracy: 0.6875
Epoch 2/10
5480/5480 [==============================] - 9s 2ms/step - loss: 0.7026 - accuracy: 0.7477
Epoch 3/10
5480/5480 [==============================] - 9s 2ms/step - loss: 0.6621 - accuracy: 0.7572
Epoch 4/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.6251 - accuracy: 0.7603
Epoch 5/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.6089 - accuracy: 0.7638
Epoch 6/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.6014 - accuracy: 0.7651
Epoch 7/10
5480/5480 [==============================] - 11s 2ms/step - loss: 0.5928 - accuracy: 0.7676
Epoch 8/10
5480/5480 [==============================] - 9s 2ms/step - loss: 0.5893 - accuracy: 0.7697
Epoch 9/10
5480/5480 [==============================] - 10s 2ms/step - loss: 0.5838 - accuracy: 0.7716
Epoch 10/10
5480/5480 [==============================] - 9s 2ms/step - loss:

In [54]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

data = input_numerical_values.values
data = data.reshape(input_numerical_values.shape[0], 1, 39, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 1, 37, 32)         128       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 18, 32)        0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 576)               0         
                                                                 
 dense_61 (Dense)            (None, 128)               73856     
                                                                 
 dense_62 (Dense)            (None, 10)                1290      
                                                                 
Total params: 75,274
Trainable params: 75,274
Non-trainable params: 0
_________________________________________________

In [55]:
model.fit(data, output1, epochs=10, batch_size=32)

Epoch 1/10
5480/5480 [==============================] - 21s 4ms/step - loss: 33.0430 - accuracy: 0.7498
Epoch 2/10
5480/5480 [==============================] - 17s 3ms/step - loss: 2.6004 - accuracy: 0.7834
Epoch 3/10
5480/5480 [==============================] - 19s 3ms/step - loss: 0.5845 - accuracy: 0.7920
Epoch 4/10
5480/5480 [==============================] - 20s 4ms/step - loss: 0.5236 - accuracy: 0.7969
Epoch 5/10
5480/5480 [==============================] - 18s 3ms/step - loss: 0.5244 - accuracy: 0.8003
Epoch 6/10
5480/5480 [==============================] - 19s 3ms/step - loss: 0.5031 - accuracy: 0.8033
Epoch 7/10
5480/5480 [==============================] - 19s 3ms/step - loss: 0.5099 - accuracy: 0.8048
Epoch 8/10
5480/5480 [==============================] - 17s 3ms/step - loss: 0.4879 - accuracy: 0.8066
Epoch 9/10
5480/5480 [==============================] - 18s 3ms/step - loss: 0.5209 - accuracy: 0.8067
Epoch 10/10
5480/5480 [==============================] - 18s 3ms/step - 

In [58]:
model = Sequential()
model.add(Conv2D(17, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 1, 37, 17)         68        
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 1, 18, 17)        0         
 2D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 306)               0         
                                                                 
 dense_65 (Dense)            (None, 64)                19648     
                                                                 
 dense_66 (Dense)            (None, 10)                650       
                                                                 
Total params: 20,366
Trainable params: 20,366
Non-trainable params: 0
_________________________________________________

In [59]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 16s 3ms/step - loss: 33.1426 - accuracy: 0.7433
Epoch 2/10
5480/5480 [==============================] - 16s 3ms/step - loss: 4.5382 - accuracy: 0.7762
Epoch 3/10
5480/5480 [==============================] - 18s 3ms/step - loss: 0.7005 - accuracy: 0.7864
Epoch 4/10
5480/5480 [==============================] - 20s 4ms/step - loss: 0.5399 - accuracy: 0.7924
Epoch 5/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.6864 - accuracy: 0.7960
Epoch 6/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5172 - accuracy: 0.7986
Epoch 7/10
5480/5480 [==============================] - 14s 3ms/step - loss: 0.5228 - accuracy: 0.8005
Epoch 8/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5154 - accuracy: 0.8014
Epoch 9/10
5480/5480 [==============================] - 14s 3ms/step - loss: 0.5038 - accuracy: 0.8023
Epoch 10/10
5480/5480 [==============================] - 14s 3ms/step - 

In [63]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 1, 37, 42)         168       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 1, 18, 42)        0         
 2D)                                                             
                                                                 
 flatten_8 (Flatten)         (None, 756)               0         
                                                                 
 dense_72 (Dense)            (None, 150)               113550    
                                                                 
 dense_73 (Dense)            (None, 10)                1510      
                                                                 
Total params: 115,228
Trainable params: 115,228
Non-trainable params: 0
_______________________________________________

In [61]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 23s 4ms/step - loss: 34.4689 - accuracy: 0.7515
Epoch 2/10
5480/5480 [==============================] - 21s 4ms/step - loss: 0.5517 - accuracy: 0.7853
Epoch 3/10
5480/5480 [==============================] - 22s 4ms/step - loss: 0.5611 - accuracy: 0.7925
Epoch 4/10
5480/5480 [==============================] - 20s 4ms/step - loss: 0.5325 - accuracy: 0.7973
Epoch 5/10
5480/5480 [==============================] - 20s 4ms/step - loss: 0.5270 - accuracy: 0.8011
Epoch 6/10
5480/5480 [==============================] - 20s 4ms/step - loss: 0.6155 - accuracy: 0.8028
Epoch 7/10
5480/5480 [==============================] - 20s 4ms/step - loss: 0.4996 - accuracy: 0.8053
Epoch 8/10
5480/5480 [==============================] - 20s 4ms/step - loss: 0.4922 - accuracy: 0.8059
Epoch 9/10
5480/5480 [==============================] - 20s 4ms/step - loss: 0.4989 - accuracy: 0.8073
Epoch 10/10
5480/5480 [==============================] - 20s 4ms/step - 

In [64]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 1, 37, 42)         168       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 1, 18, 42)        0         
 2D)                                                             
                                                                 
 flatten_9 (Flatten)         (None, 756)               0         
                                                                 
 dense_74 (Dense)            (None, 150)               113550    
                                                                 
 dense_75 (Dense)            (None, 15)                2265      
                                                                 
 dense_76 (Dense)            (None, 10)                160       
                                                     

In [65]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 24s 4ms/step - loss: 3.4085 - accuracy: 0.7427
Epoch 2/10
5480/5480 [==============================] - 22s 4ms/step - loss: 0.5925 - accuracy: 0.7672
Epoch 3/10
5480/5480 [==============================] - 23s 4ms/step - loss: 0.5582 - accuracy: 0.7736
Epoch 4/10
5480/5480 [==============================] - 22s 4ms/step - loss: 0.5432 - accuracy: 0.7798
Epoch 5/10
5480/5480 [==============================] - 24s 4ms/step - loss: 0.5274 - accuracy: 0.7839
Epoch 6/10
5480/5480 [==============================] - 23s 4ms/step - loss: 0.5215 - accuracy: 0.7876
Epoch 7/10
5480/5480 [==============================] - 23s 4ms/step - loss: 0.5155 - accuracy: 0.7890
Epoch 8/10
5480/5480 [==============================] - 23s 4ms/step - loss: 0.5093 - accuracy: 0.7903
Epoch 9/10
5480/5480 [==============================] - 21s 4ms/step - loss: 0.5024 - accuracy: 0.7929
Epoch 10/10
5480/5480 [==============================] - 22s 4ms/step - l

In [66]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 1, 37, 42)         168       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 1, 18, 42)        0         
 g2D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 756)               0         
                                                                 
 dense_77 (Dense)            (None, 10)                7570      
                                                                 
Total params: 7,738
Trainable params: 7,738
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 15s 3ms/step - loss: 46.7705 - accuracy: 0.7244
Epoch 2/10
5480/5480 [==============================] - 15s 3ms/step - loss: 13.7716 - accuracy: 0.7609
Epoch 3/10
5480/5480 [==============================] - 14s 3ms/step - loss: 9.3582 - accuracy: 0.7667
Epoch 4/10
5480/5480 [==============================] - 15s 3ms/step - loss: 11.5526 - accuracy: 0.7685
Epoch 5/10
5480/5480 [==============================] - 18s 3ms/step - loss: 7.0301 - accuracy: 0.7744
Epoch 6/10
5480/5480 [==============================] - 23s 4ms/step - loss: 4.0129 - accuracy: 0.7799
Epoch 7/10
5480/5480 [==============================] - 15s 3ms/step - loss: 1.1073 - accuracy: 0.7827
Epoch 8/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5880 - accuracy: 0.7856
Epoch 9/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5687 - accuracy: 0.7871
Epoch 10/10
5480/5480 [==============================] - 15s 3ms/step 

In [68]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 1, 37, 42)         168       
                                                                 
 flatten_11 (Flatten)        (None, 1554)              0         
                                                                 
 dense_78 (Dense)            (None, 150)               233250    
                                                                 
 dense_79 (Dense)            (None, 10)                1510      
                                                                 
Total params: 234,928
Trainable params: 234,928
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 36s 6ms/step - loss: 24.4438 - accuracy: 0.7588
Epoch 2/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.5428 - accuracy: 0.7900
Epoch 3/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.5277 - accuracy: 0.7971
Epoch 4/10
5480/5480 [==============================] - 34s 6ms/step - loss: 0.5434 - accuracy: 0.8002
Epoch 5/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.5666 - accuracy: 0.8016
Epoch 6/10
5480/5480 [==============================] - 35s 6ms/step - loss: 0.5386 - accuracy: 0.8031
Epoch 7/10
5480/5480 [==============================] - 32s 6ms/step - loss: 0.4967 - accuracy: 0.8053
Epoch 8/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.5152 - accuracy: 0.8055
Epoch 9/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.5266 - accuracy: 0.8077
Epoch 10/10
5480/5480 [==============================] - 31s 6ms/step - 

In [70]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(1, 5), activation='relu', input_shape=(1, 39, 1)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 1, 35, 42)         252       
                                                                 
 flatten_12 (Flatten)        (None, 1470)              0         
                                                                 
 dense_80 (Dense)            (None, 150)               220650    
                                                                 
 dense_81 (Dense)            (None, 10)                1510      
                                                                 
Total params: 222,412
Trainable params: 222,412
Non-trainable params: 0
_________________________________________________________________


In [71]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 31s 5ms/step - loss: 14.8356 - accuracy: 0.7593
Epoch 2/10
5480/5480 [==============================] - 30s 5ms/step - loss: 0.5434 - accuracy: 0.7929
Epoch 3/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.6604 - accuracy: 0.7983
Epoch 4/10
5480/5480 [==============================] - 31s 6ms/step - loss: 1.6356 - accuracy: 0.8007
Epoch 5/10
5480/5480 [==============================] - 30s 5ms/step - loss: 0.5176 - accuracy: 0.8052
Epoch 6/10
5480/5480 [==============================] - 32s 6ms/step - loss: 0.5881 - accuracy: 0.8056
Epoch 7/10
5480/5480 [==============================] - 31s 6ms/step - loss: 1.2413 - accuracy: 0.8039
Epoch 8/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.4901 - accuracy: 0.8071
Epoch 9/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.5528 - accuracy: 0.8085
Epoch 10/10
5480/5480 [==============================] - 31s 6ms/step - 

In [72]:
model = Sequential()
model.add(Conv2D(52, kernel_size=(1, 10), activation='relu', input_shape=(1, 39, 1)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 1, 30, 52)         572       
                                                                 
 flatten_13 (Flatten)        (None, 1560)              0         
                                                                 
 dense_82 (Dense)            (None, 150)               234150    
                                                                 
 dense_83 (Dense)            (None, 10)                1510      
                                                                 
Total params: 236,232
Trainable params: 236,232
Non-trainable params: 0
_________________________________________________________________


In [73]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 33s 6ms/step - loss: 28.6510 - accuracy: 0.7565
Epoch 2/10
5480/5480 [==============================] - 31s 6ms/step - loss: 1.1068 - accuracy: 0.7908
Epoch 3/10
5480/5480 [==============================] - 32s 6ms/step - loss: 2.4208 - accuracy: 0.7939
Epoch 4/10
5480/5480 [==============================] - 32s 6ms/step - loss: 1.2778 - accuracy: 0.7977
Epoch 5/10
5480/5480 [==============================] - 34s 6ms/step - loss: 0.5349 - accuracy: 0.7997
Epoch 6/10
5480/5480 [==============================] - 34s 6ms/step - loss: 0.6462 - accuracy: 0.7912
Epoch 7/10
5480/5480 [==============================] - 39s 7ms/step - loss: 0.5641 - accuracy: 0.7878
Epoch 8/10
5480/5480 [==============================] - 32s 6ms/step - loss: 0.5269 - accuracy: 0.7890
Epoch 9/10
5480/5480 [==============================] - 32s 6ms/step - loss: 0.5828 - accuracy: 0.7882
Epoch 10/10
5480/5480 [==============================] - 32s 6ms/step - 

In [74]:
model = Sequential()
model.add(Conv2D(52, kernel_size=(1, 2), activation='relu', input_shape=(1, 39, 1)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 1, 38, 52)         156       
                                                                 
 flatten_14 (Flatten)        (None, 1976)              0         
                                                                 
 dense_84 (Dense)            (None, 150)               296550    
                                                                 
 dense_85 (Dense)            (None, 10)                1510      
                                                                 
Total params: 298,216
Trainable params: 298,216
Non-trainable params: 0
_________________________________________________________________


In [75]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 50s 9ms/step - loss: 21.6975 - accuracy: 0.7575
Epoch 2/10
5480/5480 [==============================] - 47s 9ms/step - loss: 0.5544 - accuracy: 0.7878
Epoch 3/10
5480/5480 [==============================] - 47s 9ms/step - loss: 0.5251 - accuracy: 0.7962
Epoch 4/10
5480/5480 [==============================] - 48s 9ms/step - loss: 0.5145 - accuracy: 0.7994
Epoch 5/10
5480/5480 [==============================] - 46s 8ms/step - loss: 0.5124 - accuracy: 0.8016
Epoch 6/10
5480/5480 [==============================] - 49s 9ms/step - loss: 0.4983 - accuracy: 0.8044
Epoch 7/10
5480/5480 [==============================] - 53s 10ms/step - loss: 0.4957 - accuracy: 0.8051
Epoch 8/10
5480/5480 [==============================] - 45s 8ms/step - loss: 0.5046 - accuracy: 0.8062
Epoch 9/10
5480/5480 [==============================] - 51s 9ms/step - loss: 0.4840 - accuracy: 0.8076
Epoch 10/10
5480/5480 [==============================] - 51s 9ms/step -

In [76]:
model = Sequential()
model.add(Dense(150, input_dim=39, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_86 (Dense)            (None, 150)               6000      
                                                                 
 dense_87 (Dense)            (None, 10)                1510      
                                                                 
Total params: 7,510
Trainable params: 7,510
Non-trainable params: 0
_________________________________________________________________


In [77]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 10s 2ms/step - loss: 73.7005 - accuracy: 0.7343
Epoch 2/10
5480/5480 [==============================] - 10s 2ms/step - loss: 50.9022 - accuracy: 0.7648
Epoch 3/10
5480/5480 [==============================] - 10s 2ms/step - loss: 39.4307 - accuracy: 0.7733
Epoch 4/10
5480/5480 [==============================] - 9s 2ms/step - loss: 30.1278 - accuracy: 0.7782
Epoch 5/10
5480/5480 [==============================] - 10s 2ms/step - loss: 22.0486 - accuracy: 0.7837
Epoch 6/10
5480/5480 [==============================] - 10s 2ms/step - loss: 14.7973 - accuracy: 0.7875
Epoch 7/10
5480/5480 [==============================] - 10s 2ms/step - loss: 13.8406 - accuracy: 0.7904
Epoch 8/10
5480/5480 [==============================] - 10s 2ms/step - loss: 9.1472 - accuracy: 0.7910
Epoch 9/10
5480/5480 [==============================] - 10s 2ms/step - loss: 6.7839 - accuracy: 0.7924
Epoch 10/10
5480/5480 [==============================] - 10s 2ms/st

In [83]:
from keras.layers import Input, Dense, Attention, dot
from keras.models import Model

data = input_numerical_values.values

query_data = data[:,:19].reshape(input_numerical_values.shape[0], 1, 19)
key_data = data[:,19:28].reshape(input_numerical_values.shape[0], 1, 9)
value_data = data[:,28:].reshape(input_numerical_values.shape[0], 1, 11)

query_input = Input(shape=(1, 19))
key_input = Input(shape=(1, 9))
value_input = Input(shape=(1, 11))

query = Dense(128, activation='relu')(query_input)
key = Dense(128, activation='relu')(key_input)
value = Dense(128, activation='relu')(value_input)

attention = Attention(10)([query, key, value])
outputs = Dense(10, activation='softmax')(attention)
model = Model(inputs=[query_input, key_input, value_input], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [86]:
model.fit([query_data, key_data, value_data], output1, epochs=10, batch_size=32)

Epoch 1/10


ValueError: in user code:

    File "c:\users\devayani\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\users\devayani\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\users\devayani\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\users\devayani\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\users\devayani\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "c:\us